In [1]:
import os
import csv
import torch
import uuid
import numpy as np
import pandas as pd
import torchvision.transforms as T

from pathlib import Path
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [6]:
dataset_path = Path('./data/UCMerced_LandUse/Images')
h, w = 256, 256

transform = T.Compose([
    T.Resize((h, w)),
    T.ToTensor()
])

dataset = ImageFolder(dataset_path, transform=transform)

batch_size = 4

generator = torch.Generator()
generator.manual_seed(42)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, generator=generator)

In [7]:
class_to_idx = dataset.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}

In [8]:
toPIL = T.ToPILImage()

outputs_path = Path('./data/land_use')
os.makedirs(outputs_path, exist_ok=True)
outputs_extension = 'jpg' # 'tif', 'png', 'jpg'

csv_path = Path('./data/image_labels.csv')
csv_file = open(csv_path, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['id', 'label1', 'label2', 'label3', 'label4'])

for idx, (batch_images, batch_labels) in enumerate(dataloader):
    batch_images = [toPIL(batch_image) for batch_image in batch_images]
    batch_labels = [idx_to_class[batch_label.item()] for batch_label in batch_labels]
    
    combined_image = Image.new('RGB', (2 * w, 2 * h))

    image_id = uuid.uuid4().hex[:8]
    
    for i in range(batch_size):
        combined_image.paste(batch_images[i], (i % 2 * w, i // 2 * h))
        
    csv_writer.writerow([image_id, *batch_labels])
    combined_image.save(outputs_path / f'{image_id}.{outputs_extension}')
    
csv_file.close()

In [10]:
pd.read_csv(csv_path)

,id,label1,label2,label3,label4
0,611c6f75,denseresidential,overpass,mobilehomepark,storagetanks
1,5e6fdcff,golfcourse,parkinglot,baseballdiamond,river
2,edad1879,intersection,agricultural,denseresidential,agricultural
3,bb43d2ea,baseballdiamond,parkinglot,airplane,intersection
4,6af507b9,forest,buildings,sparseresidential,mobilehomepark
...,...,...,...,...,...
520,c55f2f01,mobilehomepark,denseresidential,airplane,tenniscourt
521,f8b35d2f,buildings,denseresidential,freeway,beach
522,9b80128b,tenniscourt,river,overpass,forest
523,1f1423ef,harbor,freeway,mobilehomepark,buildings
